# Examples of AWS Config and Commands

https://docs.aws.amazon.com/cli/latest/reference/

https://docs.aws.amazon.com/cli/index.html

json parsing
https://stedolan.github.io/jq/manual

In [ ]:
sudo apt-get update

#Insall CLI (python lib is: python3-boto3) 
sudo apt-get -y install awscli 

In [ ]:
AWS_ACCESS_KEY_ID=`tail -n +2  ~/work/new_user_credentials.csv | cut -d, -f3`
AWS_SECRRET_ACCESS_KEY=`tail -n +2  ~/work/new_user_credentials.csv | cut -d, -f4`
DEFAULT_REGION_NAME='us-east-2'
DEFAULT_OUTPUT_FORMAT='json'


echo $AWS_ACCESS_KEY_ID
echo $AWS_SECRRET_ACCESS_KEY


# Note: can be done internativly with 'aws configure'
aws configure set aws_access_key_id $AWS_ACCESS_KEY_ID
aws configure set aws_secret_access_key $AWS_SECRRET_ACCESS_KEY
aws configure set default.region $DEFAULT_REGION_NAME
aws configure set default.output $DEFAULT_OUTPUT_FORMAT

#aws configure set default.ca_bundle /path/to/ca-bundle.pem
#aws configure set region us-west-1 --profile testing
#aws configure set profile.testing2.region eu-west-1
#aws configure set preview.cloudsearch true

In [ ]:
aws  --output json ec2 describe-vpcs

In [ ]:
#Get instance id
INSTANCE_NAME="pruth-test"
#Needs try/catch b/c some instnaces don't have Tags/Names
INSTANCE_ID=`aws --output json ec2 describe-instances | jq -r '.Reservations[].Instances[] |  try if .Tags[].Key == "Name" and .Tags[].Value == "'${INSTANCE_NAME}'" then .InstanceId else "" end catch ""'`
echo INSTANCE_ID $INSTANCE_ID

In [ ]:
aws --output json ec2 stop-instances --instance-ids $INSTANCE_ID

In [ ]:
aws --output json ec2 start-instances --instance-ids $INSTANCE_ID

In [ ]:
aws --output json ec2 describe-vpcs

In [ ]:
#note about Tags being null: https://ilya-sher.org/2016/05/11/most-jq-you-will-ever-need/

#Get vpc id
VPC_NAME="vpc-chameleon-cloudconnect"

#Needs try/catch b/c some vpcs don't have Tags/Names
VPC_ID=`aws --output json ec2 describe-vpcs | jq -r '.Vpcs[] | try if .Tags[].Key == "Name" and .Tags[].Value == "'${VPC_NAME}'" then .VpcId else "" end catch ""'`

echo VPC_ID $VPC_ID

In [ ]:
aws --output json ec2 describe-vpn-gateways

In [ ]:
#Create VPG
#aws --output json ec2 describe-vpn-gateways

VPC_GATEWAY_ID=`aws --output json ec2 create-vpn-gateway --type ipsec.1 --amazon-side-asn 65001 | jq -r '.VpnGateway.VpnGatewayId'`
echo VPC_GATEWAY_ID $VPC_GATEWAY_ID
                                        
                                        

In [ ]:
#Attach VPG to VPC
aws --output json ec2 attach-vpn-gateway --vpc-id $VPC_ID --vpn-gateway-id $VPC_GATEWAY_ID


In [ ]:
ROUTE_TABLE_ID=`aws --output json ec2 describe-route-tables | jq -r '.RouteTables[] |  try if .VpcId == "'${VPC_ID}'" then .RouteTableId else "" end catch ""'`
echo ROUTE_TABLE_ID $ROUTE_TABLE_ID

In [ ]:
#Set the route to the VPG: TODO
#CHAMELEON_SUBNET="192.168.4.1/24"
#
aws --output json ec2 enable-vgw-route-propagation --gateway-id $VPC_GATEWAY_ID \
                                                   --route-table-id $ROUTE_TABLE_ID

In [ ]:
#aws --output json ec2 create-route --route-table-id $ROUTE_TABLE_ID \
#                                   --destination-cidr-block $CHAMELEON_SUBNET \
#                                   --gateway-id $VPC_GATEWAY_ID
                                   

In [ ]:
#Accept the DirectConnect Connection

#must have directconnect iam permissions
#aws --output json  directconnect describe-direct-connect-gateway-association-proposals
aws --output json  directconnect describe-connections

In [ ]:
#Get directconnect conneciton id
c
#Needs try/catch b/c some instnaces don't have Tags/Names
DIRECTCONNECT_ID=`aws --output json directconnect describe-connections | jq -r '.connections[] |  try if .connectionName == "'${DIRECTCONNECT_NAME}'" then .connectionId else "" end catch ""'`
echo DIRECTCONNECT_ID $DIRECTCONNECT_ID

In [ ]:
aws --output json directconnect confirm-connection --connection-id $DIRECTCONNECT_ID

In [ ]:
DIRECTCONNECT_NAME="pruth-chameleon-aws"
DIRECTCONNECT_STATE=""

while [ "$DIRECTCONNECT_STATE" != "available" ]
do 
    sleep 10
    #Needs try/catch b/c some instnaces don't have Tags/Names
    DIRECTCONNECT_STATE=`aws --output json directconnect describe-connections | jq -r '.connections[] |  try if .connectionName == "'${DIRECTCONNECT_NAME}'" then .connectionState else "" end catch ""'`
    echo DIRECTCONNECT_STATE $DIRECTCONNECT_STATE
done

In [ ]:
#Create DirectConnectGateway
DIRECTCONNECT_GATEWAY_NAME="dcgw-pruth-chameleon-aws"
AWS_ASN=65001

aws --output json directconnect create-direct-connect-gateway --direct-connect-gateway-name $DIRECTCONNECT_GATEWAY_NAME --amazon-side-asn $AWS_ASN

In [ ]:
#Needs try/catch b/c some instnaces don't have Tags/Names
DIRECTCONNECT_GW_ID=`aws --output json directconnect describe-direct-connect-gateways | jq -r '.directConnectGateways[] |  try if .directConnectGatewayName == "'${DIRECTCONNECT_GATEWAY_NAME}'" then .directConnectGatewayId else "" end catch ""'`
echo DIRECTCONNECT_GW_ID $DIRECTCONNECT_GW_ID

In [ ]:
#Create VirtualPrivateGateway
VIRTUAL_INTERFACE_NAME="vif-pruth-chameleon-aws"
AWS_VLAN=3297
I2_ASN=55038
AWS_AUTHKEY="BgpAuthPass"
AWS_ADDR=192.168.2.1/24
I2_ADDR=192.168.2.2/24
MTU=9001
aws --output json directconnect create-private-virtual-interface \
                                          --connection-id $DIRECTCONNECT_ID \
                                          --new-private-virtual-interface virtualInterfaceName=${VIRTUAL_INTERFACE_NAME},vlan=${AWS_VLAN},asn=${I2_ASN},authKey=${AWS_AUTHKEY},mtu=${MTU},amazonAddress=${AWS_ADDR},customerAddress=${I2_ADDR},directConnectGatewayId=${DIRECTCONNECT_GW_ID}
                                          
                                          

In [ ]:
#Associate VPGW with DirectConnect GW

aws --output json  directconnect create-direct-connect-gateway-association --direct-connect-gateway-id $DIRECTCONNECT_GW_ID \
                                                                           --virtual-gateway-id $VPC_GATEWAY_ID

In [ ]:
#DELETE ALL
